# Models Exploration using CUB dataset

## References
* [Transfer Learning with Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub)
* [`tf.keras.utils.image_dataset_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory)
* [Limiting GPU Memory Growth](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth)

## Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import datetime
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub

In [2]:
def limit_memory_growth(limit=True):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, limit)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

In [3]:
limit_memory_growth()

1 Physical GPUs, 1 Logical GPUs


## Utility

In [4]:
def plot_predictions(
    image_batch,
    predicted_class_names,
):
    plt.figure(figsize=(10,9))
    plt.subplots_adjust(hspace=0.5)
    for n in range(30):
        plt.subplot(6,5,n+1)
        plt.imshow(image_batch[n])
        plt.title(predicted_class_names[n])
        plt.axis('off')
    _ = plt.suptitle("Predictions")

def plot_images(
    ds,
    class_names,
):
    plt.figure(figsize=(10, 10))
    for images, labels in ds.tjake(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")
    
def get_timestamp():
    return datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

## Dataset

In [5]:
def preprocess_fun( x, y, preprocess_input ):
    print(type(x))
    print(x.shape)
    print(type(y))
    print(y.shape)
    return (x, y)

def build_dataset(
    data_dir = '/mnt/cub/CUB_200_2011/images',
    batch_size = 64,
    image_size = (299,299),
    preprocess_input = None,
    # normalization = True,
):
   
    train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        batch_size = batch_size,
        validation_split = 0.2,
        image_size = image_size,
        subset = "both",
        shuffle = True, # default but here for clarity
        seed=42,
    )
    '''
    datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        validation_split=0.2,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.15,
        zoom_range=0.1,
        channel_shift_range=10.,
        horizontal_flip=True,
    )
    
    train_ds = datagen.flow_from_directory(
        data_dir,
        target_size=(299,299),
        batch_size=batch_size,
        subset='training',
        shuffle=True,
    )

    val_ds = datagen.flow_from_directory(
        data_dir,
        target_size=(299,299),
        batch_size=batch_size,
        subset='validation',
        shuffle=True,
    )
    '''
    


    
    '''

    # Use model specific preprocessing function
    if preprocess_input:
        train_ds.map(lambda x, y: (preprocess_input(x), y))
        val_ds.map(lambda x, y: (preprocess_input(x), y))
    else:
        # normalization_layer = layers.Rescaling(
        #     1./255,
        #     name="normalization_layer",
        # )
        # train_ds.map(lambda x, y: (normalization_layer(x)-0.5, y))
        # val_ds.map(lambda x, y: (normalization_layer(x)-0.5, y))
        pass
    '''
    
    # Retrieve class names
    # (can't do this after converting to PrefetchDataset?)
    class_names = train_ds.class_names
    
    # Prefetch images
    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    # apply preprocessing function
    train_ds.map(
        lambda x, y: preprocess_fun(x, y, preprocess_input)
    )
    
    return (train_ds, val_ds, class_names)

## Enumerate Models to test

In [6]:
base_models_metadata = [
    # ('https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4', 224),
    # ('https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4', 299),
    # ('https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5', 299),
    (tf.keras.applications.Xception, 299, tf.keras.applications.xception.preprocess_input),
    # (tf.keras.applications.resnet.ResNet101, 224),
    # (tf.keras.applications.ResNet50, 224),
    # (tf.keras.applications.InceptionResNetV2, 299),
    # (tf.keras.applications.efficientnet_v2.EfficientNetV2B0, 224)
]

def get_model_name( model_handle ):
    
    if callable(model_handle):
        return f'keras.applications.{model_handle.__name__}'
    else:
        split = model_handle.split('/')
        return f'{split[-5]}.{split[-4]}.{split[-3]}'
    

## Model Building

In [7]:
def build_base_model_layer(
    model_handle,
    name="base_model_layer",
):
    if callable(model_handle):
        base_model_layer = model_handle(
            include_top=False,
            weights='imagenet',
            pooling = 'avg',
            # trainable=False,
        )
    else:
        base_model_layer = hub.KerasLayer(
            model_handle,
            name=name,
            trainable = False, # default but here for clarity
        )
    # print(base_model_layer.get_config())
    # print(base_model_layer.summary())
    return base_model_layer

def test(x):
    tf.print(x.shape)
    return x

def build_model(
    base_model_metadata,
    num_classes,
    dropout,
):
    model_handle, input_dimension, preprocess_input = base_model_metadata

    model = Sequential([
        # layers.Lambda(preprocess_input),
        build_base_model_layer(
            model_handle,
        ),
        layers.Dense(
            num_classes,
            # activation = 'softmax',
        ),
        layers.Dropout(dropout),
        layers.Activation("softmax", dtype="float32"),
    ])
    
    # model.build(
    #     (batch_size,) + (input_dimension, input_dimension) + (3,),
    # )
    
    return model

## Build and run all models

In [8]:
# Hyperparameters
batch_size = 64
max_epochs = 5
# dropout = 0.4
dropout = 0.4
learning_rate = 0.0001

# Directory for logs
base_log_dir = "models_cub_logs"

# for each base model
for base_model_metadata in base_models_metadata:
    
    model_handle, input_dimension, preprocess_input = base_model_metadata

    image_size = (input_dimension, input_dimension)
    
    # Build dataset/pipeline
    train_ds, val_ds, class_names = build_dataset(
        batch_size = batch_size,
        image_size = image_size,
        preprocess_input = preprocess_input,
    )
    
    # Build model
    model = build_model(
        base_model_metadata,
        len(class_names),
        dropout,
    )
    
    # Compile model
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
        # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        loss=tf.keras.losses.CategoricalCrossentropy(
            # from_logits=True,
        ),
        metrics=[
            'accuracy',
            # tf.keras.metrics.SparseCategoricalAccuracy(),
            # tf.keras.metrics.SparseCategoricalCrossentropy(from_logits=True),
            # tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="Top3"),
            # tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name="Top10"),
        ],
    )
    
    # Logging
    model_id = get_model_name(model_handle)
    log_dir = os.path.join( base_log_dir, model_id )
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
    )
    
    # Early stopping
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        # monitor='val_sparse_categorical_accuracy',
        monitor='accuracy',
        patience=5,
        min_delta=0.001,
    ),
    
    print()
    print(model_id)
    
    # Train
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=max_epochs,
        callbacks=[
            tensorboard_callback,
            early_stopping_callback,
        ]
    )
    
    # Save model
    model.save(os.path.join(log_dir, 'final_model' ))    
    

Found 11788 files belonging to 200 classes.
Using 9431 files for training.
Using 2357 files for validation.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 299, 299, 3)
<class 'tensorflow.python.framework.ops.Tensor'>
(None,)

keras.applications.Xception
Epoch 1/5


ValueError: in user code:

    File "/home/charlescoult/.conda/envs/fungi/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/charlescoult/.conda/envs/fungi/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/charlescoult/.conda/envs/fungi/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/charlescoult/.conda/envs/fungi/lib/python3.10/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/charlescoult/.conda/envs/fungi/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/home/charlescoult/.conda/envs/fungi/lib/python3.1